#  Pygame : Clase 2


## Game loop
Cada vez que inicimos un juego, podemos pensar que tiene que repetir la misma serie de acciones desde que oprimos _start_ hasta que finaliza, a este ciclo se le conoce como __game loop__

Los pasos que se nos vienen a la cabeza son :
1. Pantalla de inicio
2. Correr un juego nuevo (dependerá de si hay puntos de guardado)
3. Pantalla de fin

Vamos a ver como sería escrito en python.

In [ ]:
# Por "norma" llamaremos a este archivo main.py
import pygame as pg  # el 'as' sirve para hacer abreviaciones
from configuracion import *
# el * nos sirve para usar otro archivo sin tener que hacer referencia directa


class Juego:
    def __init__(self):  # Se inicializa todo lo necesario para que corra
        pg.init()
        # Las variables que están en MAYÚSCULAS se colocarán en un archivo adicional llamado configuracion.py
        self.ventana = pg.display.set_mode((ANCHO, ALTO))
        pg.display.set_caption(TITULO)
        self.clock = pg.time.Clock()
        self.corriendo = True

    def nuevo(self):  # inicia un nuevo juego
        # Aquí colocaremos todos los elementos del juego
        self.run()

    def run(self):  # loop del juego
        self.jugando = True
        while self.jugando:
            self.clock.tick(FPS)
            self.eventos()
            self.actualizar()
            self.draw()

    def eventos(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:  # Se sale del juego
                if self.jugando:
                    self.jugando = False
                self.corriendo = False

    def actualizar(self):  # Revisará que ha cambiado y tomará decisiones
        pass

    def draw(self):  # Re dibuja la pantalla
        pass

    def pantalla_inicio(self):
        pass

    def pantalla_fin(self):
        pass


juego = Juego()
juego.pantalla_inicio()
while juego.corriendo:
    juego.nuevo()
    juego.pantalla_fin()

pg.quit()
    

In [ ]:
# configuracion.py
ANCHO = 500
ALTO = 300
TITULO = 'GAME LOOP'
FPS = 60

De ahora en adelante todos los juegos y ejemplos hechos aquí tendrán la misma estructura.

## Sprites
Los _sprites_ son imagenes en 2D que se componen de bits (bitmap) y que erán muy utiles en los juegos de los 80's y 90's por su facilidad de ser cargadas y renderizadas en los juegos. Estas imagenes representan personajes, plataformas y en general elementos del entorno.

![contra](https://external-content.duckduckgo.com/iu/?u=https%3A%2F%2Ffew4th.files.wordpress.com%2F2007%2F10%2Foriginal_contra.jpg&f=1&nofb=1)

Pygame cuenta con modulo especifico para manejar este tipo de objetos, y es recomendable hacerlo de la forma que está en la documentación, pues ya está bien optimizada para su ejecución.

Antes de introducir imagenes, vamos a usar las propiedades del modulo __pygame.sprite.Sprite__ para hacer un personaje que se mueva y salte por la pantalla.

## Saltos de personajes
Cuando jugamos videojuegos, se nos hacen "realistas" o mas sencillos de manejar aquellos personajes cuyos mundos tienen las misma fisica que el nuestro, por ejemplo, cuando un personaje choca contra una pared esperamos que se detenga, si este siguiera con su camino rompería con el ambiente. Es por eso que los saltos en los video juegos, se hacen emulando las fisicas del mundo real (depende del juego) 

In [ ]:
# lo primero que modificaremos será el aarchivo configuracion.py
ANCHO = 500
ALTO = 300
TITULO = 'SALTOS'
FPS = 60


# Colores
NEGRO = (0, 0, 0)
BLANCO = (255, 255, 255)
ROJO = (255, 0, 0)
VERDE = (0, 255, 0)
AZUL = (0, 0, 255)

# Caracteristicas del jugador
ANCHO_JUGADOR = 50
ALTO_JUGADOR = 50
GRAVEDAD = 0.8
FRICCCION = -0.12
ACC_JUGADOR = 0.5


# añadimos dos plataformas
LISTA_PLATAFORMAS = [(0, ALTO - 40, ANCHO, 40),
                     (ANCHO / 2 - 50, ALTO / 2, 100, 20),
                     ]

In [ ]:
# despues crearemos un nuevo archivo que contendrá el codigo de nuestros sprites
# llameremos a este archivo sprites.py
import pygame as pg
from configuracion import *
vector = pg.math.Vector2  # sirve para manejar el movimiento en 2D

# para poder usar laas funcionalidades del modulo, tenemos que heredar de él
class Jugador(pg.sprite.Sprite):
    def __init__(self, juego):
        pg.sprite.Sprite.__init__(self)
        self.juego = juego  # Hacemos una referencia al juego
        # la referencia del juego nos sirve para manejar variables de este
        self.image = pg.Surface((ANCHO_JUGADOR, ALTO_JUGADOR))
        self.image.fill(AZUL)
        self.rect = self.image.get_rect()
        self.rect.center = (ANCHO / 2, ALTO / 2)
        self.pos = vector(ANCHO / 2, ALTO / 2)
        self.vel = vector(0, 0)
        self.acc = vector(0, 0)

    def jump(self):
        # salta si solo si está en una plataforma
        self.rect.y += 1  # corre en 1 pixel a ver si colisiona
        colision = pg.sprite.spritecollide(
            self, self.juego.platafaformas, False)
        self.rect.y -= 1
        if colision:
            self.vel.y = -20

    def update(self):  # importante que se llame update, no valen traducciones
        # ponemos la gravedad positiva por como maneja pygame las coordenadas
        self.acc = vector(0, GRAVEDAD)

        keys = pg.key.get_pressed()
        if keys[pg.K_LEFT]:
            self.acc.x = -ACC_JUGADOR

        if keys[pg.K_RIGHT]:
            self.acc.x = ACC_JUGADOR

        self.acc.x += self.vel.x * FRICCCION
        self.vel += self.acc  # equación de la velocidad
        if abs(self.vel.x) < 0.5:
            self.vel.x = 0
        self.pos += self.vel + 0.5 * self.acc  # equación de la posición

        # vuelve a aparecer en el otro lado de la pantalla
        if self.pos.x > ANCHO:
            self.pos.x = 0
        if self.pos.x < 0:
            self.pos.x = ANCHO
        self.rect.midbottom = self.pos  # actualizamos su posición


class Plataforma(pg.sprite.Sprite):
    # w y h hacen referencia a ancho y alto (en ingles)
    def __init__(self, x, y, w, h):
        pg.sprite.Sprite.__init__(self)
        self.image = pg.Surface((w, h))
        self.image.fill(VERDE)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y



In [ ]:
# en el archivo main.py
import pygame as pg
from configuracion import *
from sprites import *


class Juego:
    def __init__(self):
        pg.init()
        self.ventana = pg.display.set_mode((ANCHO, ALTO))
        pg.display.set_caption(TITULO)
        self.clock = pg.time.Clock()
        self.corriendo = True

    def nuevo(self):
        # inicia un nuevo juego
        self.puntaje = 0
        self.all_sprites = pg.sprite.Group()
        self.platafaformas = pg.sprite.Group()
        self.jugador = Jugador(self)
        self.all_sprites.add(self.jugador)

        # añadimos cada una de las plataformas
        for plat in LISTA_PLATAFORMAS:
            p = Plataforma(*plat)  # el * sirve para descomponer cada elemento
            self.all_sprites.add(p)
            self.platafaformas.add(p)

        self.run()

    def run(self):
        # loop del juego
        self.jugando = True
        while self.jugando:
            self.clock.tick(FPS)
            self.eventos()
            self.actualizar()
            self.draw()

    def actualizar(self):
        # le dice a todos los sprites del grupo que ejecuten el metodo update()
        self.all_sprites.update()
        if self.jugador.vel.y > 0: # si no estamos cayendo
            # definimos un operador  bool que mira si el jugador cohcó con alguna de las plataformas
            colision = pg.sprite.spritecollide(
                self.jugador, self.platafaformas, False)
            if colision:
                if self.jugador.pos.y < colision[0].rect.bottom:
                    self.jugador.pos.y = colision[0].rect.top
                    self.jugador.vel.y = 0
                    self.jugador.rect.midbottom = self.jugador.pos  # corrige los micro saltos

    def eventos(self):
        for event in pg.event.get():
            if event.type == pg.QUIT:  # Se sale del juego
                if self.jugando:
                    self.jugando = False
                self.corriendo = False
        
            if event.type == pg.KEYDOWN:
                if event.key == pg.K_SPACE:
                    self.jugador.jump()

    def draw(self):
        self.ventana.fill(BLANCO)
        self.all_sprites.draw(self.ventana)
        pg.display.flip()

    def pantalla_inicio(self):
        pass

    def pantalla_fin(self):
        pass


juego = Juego()
juego.pantalla_inicio()
while juego.corriendo:
    juego.nuevo()
    juego.pantalla_fin()

pg.quit()


El resultado debe verse así:
![saltos](saltos.png)

Se maneja con las flechas y el espacio

### Bibliografía
Este documento está fuertemente inspirado por la serie de tutoriales hechos por el canal "KidsCanCode" en youtube, la lista de reproducción de sus tutoriales es : https://www.youtube.com/playlist?list=PLsk-HSGFjnaH5yghzu7PcOzm9NhsW0Urw 